In [2]:
%reload_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px

from manofunkcijos import *

import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver # naršyklės kontroleris
from selenium.webdriver.chrome.options import Options #Naršyklės
# pip install beautifulsoup4 selenium
import time

In [8]:
url = 'https://ec.forexprostools.com/'

page = requests.get(url)
source = page.content

bs = BeautifulSoup(source, 'html.parser')

straipsniai = bs.find_all('td', {'class':'theDay'})
print(len(straipsniai))

0


In [10]:
opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.varle.lt/dronai/'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [11]:
bs = BeautifulSoup(source, 'html.parser')

In [16]:
sk = bs.find('div', {'class': 'GRID_ITEM'}).find('a')
print(sk)
print('::REM::')
print(sk['href'])
img = sk.find('img')
print(img['title'])

<a href="/dronai/dronas-dji-mini-2-se-fly-more-combo--23881156.html">
<img alt="Nuotrauka Nr.1" class="product-img primary-img" height="195" loading="lazy" src="https://www.varle.lt/static/uploads/products/235x195/404/bun/bundle-fly-more-dji-mini-2-se-mini-drone.jpg" title="Dronas DJI Mini 2 SE Fly More Combo" width="235"/>
<img alt="Nuotrauka Nr.2" class="product-img secondary-img" height="195" loading="lazy" src="https://www.varle.lt/static/uploads/products/235x195/404/bun/bundle-fly-more-dji-mini-2-se-mini-drone_Wo609zb.jpg" title="Dronas DJI Mini 2 SE Fly More Combo" width="235"/>
</a>
::REM::
/dronai/dronas-dji-mini-2-se-fly-more-combo--23881156.html
Dronas DJI Mini 2 SE Fly More Combo


In [7]:
opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
url = 'https://kauno.diena.lt/'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()
bs = BeautifulSoup(source, 'html.parser')

In [8]:
straipsniai = bs.find_all('div', {'class':'category-block-row grid-12 alpha omega unscore'})
print(len(straipsniai))

41


In [9]:
strp_pav = []
strp_nuor = []
ant_ilg = []

for s in straipsniai:
    i = s.find('div', {'class':'articles-list-title-wrapper'})
    if i is not None:
        t = i.find('a', {'class':'articles-list-title'})
        strp_pav.append(t.text)
        ant_ilg.append(len(t.text.split()))
        x = t['href']
        c = x.count('http')
        if c !=0:
            strp_nuor.append(x)
        else:
            strp_nuor.append(url+x)
avg_ant_ilg = sum(ant_ilg)/len(ant_ilg)
print(strp_pav,'\n',strp_nuor, '\n', ant_ilg, '\n', avg_ant_ilg)

['2024-ųjų Seimo rinkimai: laimėjo socdemai, antri – TS-LKD, treti – „Nemuno Aušra“ ', 'Į antrąjį Seimo rinkimų turą patekę kandidatai (apžvalga)', 'Judrioje Aleksoto gatvėje – laikini eismo ribojimai', 'Septyniose apygardose bus perskaičiuoti balsai, Panemunėje gali būti trys kandidatai ', 'Kauniečiai noriai balsavo Seimo rinkimuose: kai kur – stebinantis aktyvumas', 'Vytėnų gyventojas: šviesoforas neveikia, neįmanoma išvažiuoti', '„Lidl Lietuva“: ir toliau didinsime parduotuvių bei logistikos centro darbuotojų atlyginimus', 'Akibrokštas privačiai greitajai pagalbai: 50 įdarbintų žmonių valstybei neįdomu?', 'Vyriausybė nepritaria, kad baigę gyvulininkystę teiktų sveikatos paslaugas su gyvūnais', 'Šeima – tarsi universitetas, kuriame mokomės iš artimųjų', 'Verslas apie lūkesčius po Seimo rinkimų: stabilumas yra aukso vertės', 'Rezultatų laukimas rinkimų štabuose: kas puolė ieškotis darbo, o kas – valdyti', 'Iš išaugtų vaikiškų drabužių – prisiminimų šiltukai', 'Mainų platformos įkūrėja

In [14]:
SDB = sqlite3.connect('web_scrap.db') # jei neegzistuoja, bus sukurta nauja sqlite DB
Cs = SDB.cursor()
# if not exsists ignoruoja lenteles kurima jai tokia yra
sql = '''create table if not exists kauno_diena_lt
(
    Pavadinimas text not null,
    Nuoroda text not null,
    Žodžiai_pav real not null
)'''

Cs.execute(sql)

sql_template = '''insert into kauno_diena_lt values (?,?,?)'''
# data = [(gamintojas, kaina, rida, amzius,kiekis)]
# Cs.executemany(sql_template, data)
for pav, nuor, zodz in zip(strp_pav,strp_nuor,ant_ilg):
    sql_template = '''insert into kauno_diena_lt values (?,?,?)'''
    Cs.execute(sql_template, (pav, nuor, zodz))

SDB.commit()

SDB.close()

In [13]:
SDB.close()

In [2]:
opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.delfi.lt/'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()
bs = BeautifulSoup(source, 'html.parser')

In [3]:
straipsniai = bs.find_all('div', {'class':'block-type-102-headline__title'})
print(len(straipsniai))

71


In [5]:
strp_pav = []
strp_nuor = []
ant_ilg = []

for s in straipsniai:
    if s is not None:
        t = s.find('a')
        strp_pav.append(t.text)
        ant_ilg.append(len(t.text.split()))
        x = t['href']
        c = x.count('http')
        if c !=0:
            strp_nuor.append(x)
        else:
            strp_nuor.append(url+x)
avg_ant_ilg = sum(ant_ilg)/len(ant_ilg)
print(strp_pav,'\n',strp_nuor, '\n', ant_ilg, '\n', avg_ant_ilg)

['Per ketverius metus – kone kardinaliai pasikeitusi situacija: Karbauskis turi dėl ko pykti ant Žemaitaičio', 'Jansonas įvertino Seimo rinkimų rezultatus: buvo sukurtas Frankenšteinas ', 'Karo ekspertai: rusai įvykdė baisų nusikaltimą ', 'Rusai sunaikino trečdalį Ukrainos energetinių pajėgumų – kaip Lietuva pasiruošė tokiai atakai', 'Užkalnis. Kokie žmonės, tokie ir rinkimai', '„Bolt“ pavežėjai kreipsis į teismą: teigia, kad po Naujųjų daug kam nebeapsimokės dirbti', 'Naujas anticiklonas netrukus atneš malonių pokyčių', 'Jei savo kelyje sutiksi Budą – nužudyk jį', 'Dviejose savivaldybėse pasiektas epideminis sergamumo gripu lygis, 7 žmonės pateko į reanimaciją', 'Masiniai atleidimai įsisiūbuoja – kam ruoštis', 'Pikantiškoje fotosesijoje įsiamžinusią Kristiną Meseguer vyras įvertino ispanišku išsireiškimu: lietuviai – konservatyvesni', 'Pašniukštinėkite, kaip balsavo gyventojai jūsų aplinkoje: rezultatai pagal rinkimų apylinkes', 'Kasčiūno pralenktas Landsbergis to nesureikšmina ir net

In [6]:
SDB = sqlite3.connect('web_scrap.db') # jei neegzistuoja, bus sukurta nauja sqlite DB
Cs = SDB.cursor()
sql = 'Drop table if exists delfilt'
Cs.execute(sql)
# if not exsists ignoruoja lenteles kurima jai tokia yra
sql = '''create table if not exists delfilt
(
    Pavadinimas text not null,
    Nuoroda text not null,
    Žodžiai_pav real not null
    
)'''

Cs.execute(sql)

sql_template = '''insert into delfilt values (?,?,?)'''
# data = [(gamintojas, kaina, rida, amzius,kiekis)]
# Cs.executemany(sql_template, data)
for pav, nuor, zodz in zip(strp_pav,strp_nuor,ant_ilg):
    sql_template = '''insert into delfilt values (?,?,?)'''
    Cs.execute(sql_template, (pav, nuor, zodz))

SDB.commit()

SDB.close()

In [54]:
SDB.close()

In [ ]:
auto_skelbimai = []



while True:
    try:
        opcijos = Options()
        opcijos.add_argument('--incognito')
        driver = webdriver.Chrome(options=opcijos)
        # maxp = int(check_if_value_exists_or_max('web_scrap.db','autopliuslt','Page'))
        # maxp = 105
        for p in range(1000):
            if p != 0:
                page = p
            else:
                page = p+1
            print(page)
            url = f'https://autoplius.lt/skelbimai/naudoti-automobiliai?page_nr={page}'
            driver.get(url)
            time.sleep(5)
            source = driver.page_source
            bs = BeautifulSoup(source, 'html.parser')
            straipsniai = bs.find('div', {'class':'auto-lists lt'}).find_all('a')


            for s in straipsniai:
                if s is not None:
                    url = s['href']
                    if not check_if_value_exists_or_max('web_scrap.db','autopliuslt','URL',url):
                        driver.get(url)
                        
                        source = driver.page_source
                        bs = BeautifulSoup(source, 'html.parser')
                        
                        skelbimas = bs.find_all('div', {'class':'parameter-row'})
                        marke = bs.find_all('li', {'class':'crumb'})[2].text.strip()
                        modelis = bs.find_all('li', {'class':'crumb'})[3].text.strip()
                        kaina = bs.find('div', {'class':'price'}).text.strip().split('\n')[0].replace(' ','')[:-1]
                        skelb = {}
                        skelb['Markė']=marke
                        skelb['Modelis']=modelis
                        skelb['Kaina']=kaina
                        for s in skelbimas:
                            x = s.find_all('div', {'class':'parameter-value'})
                            p = s.find_all('div', {'class':'parameter-label'})
                            for i, z in zip(x,p):
                                skelb[z.text.strip()]=i.text.strip()
                        skelb['URL'] = url
                        skelb['Page'] = page
                        auto_skelbimai.append(skelb)
                        insert_into_db(skelb,'web_scrap.db','autopliuslt')


        driver.close()
    except Exception as e:
        print(e)
        driver.close()
    time.sleep(60)


In [205]:
print(auto_skelbimai)

[{'Markė': 'BMW', 'Modelis': '428 Gran Coupe', 'Kaina': '18000', 'Pirma registracija': '2015', 'Rida': '199 000 km', 'Variklis': '1997 cm³, 245 AG (180kW)', 'Kuro tipas': 'Benzinas', 'Kėbulo tipas': 'Kupė (Coupe)', 'Durų skaičius': '4/5', 'Varantieji ratai': 'Visi varantys (4х4)', 'Pavarų dėžė': 'Automatinė', 'Klimato valdymas': 'Klimato kontrolė', 'Spalva': 'Balta', 'Sėdimų vietų skaičius': '5', 'Pirmosios registracijos šalis': 'JAV', 'Kėbulo numeris (VIN)': 'WBA4A7C5...\n            Rodyti', 'CO₂ emisija, g/km': '~ 147 g/km', 'Taršos mokestis': '~ 40.47 €'}, {'Markė': 'BMW', 'Modelis': '530', 'Kaina': '26900', 'Pirma registracija': '2019', 'Rida': '46 000 km', 'Variklis': '1998 cm³, 252 AG (185kW)', 'Kuro tipas': 'Benzinas', 'Kėbulo tipas': 'Sedanas', 'Durų skaičius': '4/5', 'Pavarų dėžė': 'Automatinė', 'Spalva': 'Pilka / sidabrinė', 'Tech. apžiūra iki': '2026-07', 'Ratlankių skersmuo': 'R20', 'Sėdimų vietų skaičius': '5', 'CO₂ emisija, g/km': '~ 126 g/km', 'Taršos mokestis': '0 €'},

In [150]:
straipsniai = bs.find('div', {'class':'auto-lists lt'}).find_all('a')
print(len(straipsniai))

AttributeError: 'NoneType' object has no attribute 'find_all'

In [111]:
strp_nuor = []


for s in straipsniai:
    if s is not None:
        strp_pav.append(p)
        x = s['href']
        strp_nuor.append(x)
print(strp_nuor)

['https://autoplius.lt/skelbimai/bmw-428-gran-coupe-2-0-l-kupe-coupe-2015-benzinas-26837939.html', 'https://autoplius.lt/skelbimai/bmw-530-2-0-l-sedanas-2019-benzinas-26808537.html', 'https://autoplius.lt/skelbimai/bmw-530-2-0-l-sedanas-2021-benzinas-26749453.html', 'https://autoplius.lt/skelbimai/bmw-530-3-0-l-sedanas-2004-benzinas-26675965.html', 'https://autoplius.lt/skelbimai/fiat-500x-2-4-l-visureigis-krosoveris-2016-benzinas-dujos-26862175.html', 'https://autoplius.lt/skelbimai/bmw-320-gran-turismo-2-0-l-hecbekas-2016-dyzelinas-26830873.html', 'https://autoplius.lt/skelbimai/mazda-cx-5-2-0-l-visureigis-krosoveris-2017-benzinas-26740367.html', 'https://autoplius.lt/skelbimai/mercedes-benz-clk320-3-2-l-kupe-coupe-2002-benzinas-dujos-26445796.html', 'https://autoplius.lt/skelbimai/bmw-520-2-0-l-universalas-2006-dyzelinas-26806059.html', 'https://autoplius.lt/skelbimai/volkswagen-amarok-2-0-l-pikapas-2014-dyzelinas-26615729.html', 'https://autoplius.lt/skelbimai/land-rover-discovery-

In [194]:
opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
url = 'https://autoplius.lt/skelbimai/bmw-428-gran-coupe-2-0-l-kupe-coupe-2015-benzinas-26837939.html'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()
bs = BeautifulSoup(source, 'html.parser')

In [190]:
skelbimas = bs.find_all('div', {'class':'parameter-row'})
marke = bs.find_all('li', {'class':'crumb'})[2].text.strip()
modelis = bs.find_all('li', {'class':'crumb'})[3].text.strip()
kaina = bs.find('div', {'class':'price'}).text.strip().split('\n')[0].replace(' ','')[:-1]

In [195]:
c = []
c.append(kaina)
print(c)

['48500']


In [199]:
skelb = {}
skelb['Markė']=marke
skelb['Modelis']=modelis
for s in skelbimas:
    x = s.find_all('div', {'class':'parameter-value'})
    p = s.find_all('div', {'class':'parameter-label'})
    for i, z in zip(x,p):
        skelb[z.text.strip()]=i.text.strip()
print(skelb)

{'Markė': 'BMW', 'Modelis': '530', '2021-11': '2021-11', '25 000 km': '25 000 km', '1998 cm³, 262 AG (193kW)': '1998 cm³, 262 AG (193kW)', 'Benzinas': 'Benzinas', 'Sedanas': 'Sedanas', '4/5': '4/5', 'Visi varantys (4х4)': 'Visi varantys (4х4)', 'Automatinė': 'Automatinė', 'Klimato kontrolė': 'Klimato kontrolė', 'Pilka / sidabrinė': 'Pilka / sidabrinė', '2026-06': '2026-06', 'R19': 'R19', '1680': '1680', '5': '5', 'JAV': 'JAV', 'WBA13BJ0...\n            Rodyti': 'WBA13BJ0...\n            Rodyti', 'THPRH...\n                Rodyti': 'THPRH...\n                Rodyti', 'Euro 6': 'Euro 6', '135 g/km': '135 g/km', '20.23 €': '20.23 €', '7': '7', '5,9': '5,9'}
